In [2]:
import nibabel as nib
from nibabel.affines import apply_affine
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy.io as sio
from scipy.spatial import distance

### Load Brain Atlas Data and Metadata

Loads the brain atlas NIfTI image and its metadata, along with the ROI lookup table (LUT) for region labels.

- **Steps**:
  1. Load the atlas image (`aparc+aseg.nii.gz`) with `nibabel`.
  2. Extract the header and affine matrix for metadata and spatial transformations.
  3. Convert the atlas data to a NumPy array for manipulation.
  4. Load the `desikanKilliany.csv` file as `atlasLUT` for ROI information.

- **Output**: `dkatlas` (NIfTI image), `hdr` (header), `affine` (transformation matrix), `data` (array), `atlasLUT` (ROI lookup table).

In [3]:
dkatlas = nib.load('../Data/aparc+aseg.nii.gz')
hdr = dkatlas.header
affine = dkatlas.affine
data = np.asarray(dkatlas.dataobj)
atlasLUT = pd.read_csv('../Data/desikanKilliany.csv')

FileNotFoundError: No such file or no access: '../Data/aparc+aseg.nii.gz'

### Extract and Map ROI Coordinates

This block extracts ROI voxel coordinates, transforms them to real-world space using the affine matrix, and maps them with labels and colors from `atlasLUT`.

- **Steps**:
  1. Extract voxel coordinates of ROIs using `np.where`.
  2. Apply affine transformation for real-world (RASmm) coordinates.
  3. Create a DataFrame with coordinates and map ROI labels via `atlasLUT`.

- **Output**: `dkatlas_coords` DataFrame with `x`, `y`, `z` coordinates, ROI numbers, labels, and colors.

In [ ]:
# Get the coordinates of all roi values in data
coords = np.column_stack(np.where(np.isin(data, atlasLUT.roiNum)))

# Apply the affine transformation to voxel coordinates
RASmm = apply_affine(affine, coords)

# Create a DataFrame from the transformed coordinates
dkatlas_coords = pd.DataFrame(RASmm, columns=['x', 'y', 'z'])

# Map the roi values to the coordinates
dkatlas_coords['roiNum'] = data[coords[:, 0], coords[:, 1], coords[:, 2]]

# Assign color to each ROI by matching the roiNum to the LUT
dkatlas_coords = dkatlas_coords.merge(atlasLUT, on='roiNum', how='left')

### 3D Scatter Plot of ROI Coordinates

Creates an interactive 3D scatter plot of the atlas coordinates with ROIs color-coded.

- **Steps**:
  1. Use `plotly.express.scatter_3d` to plot `dkatlas_coords` DataFrame.
  2. Set `x`, `y`, and `z` for coordinates and color points by `roi` for easy visualization of different regions.

- **Output**: An interactive 3D plot displaying ROI positions.

In [ ]:
fig = px.scatter_3d(dkatlas_coords, x='x', y='y', z='z', color='roi')
fig.show()

### Add MNI Atlas Channel Positions to 3D Plot

Loads MNI atlas channel coordinates and adds them as markers to an existing 3D plot.

- **Steps**:
  1. Load `MNI_atlas.mat` and convert `ChannelPosition` to `mni_df` DataFrame with `x`, `y`, and `z` columns.
  2. Add a new 3D scatter trace to `fig` for MNI channel positions, with customized marker size and color for distinction.
  3. Display the updated plot with both atlas coordinates and MNI channel positions.

- **Output**: A combined interactive 3D plot showing both ROI and MNI channel positions.

In [ ]:
mni_atlas = sio.loadmat('../Data/MNI_atlas.mat')
mni_df = pd.DataFrame(mni_atlas['ChannelPosition'], columns=['x', 'y', 'z'])

In [ ]:
fig.add_trace(go.Scatter3d(
    x=mni_df['x'],
    y=mni_df['y'],
    z=mni_df['z'],
    mode='markers',
    marker=dict(size=5, color='black'),  # Customize size and color for distinction
    name='Electrodes'
))

# Show the combined plot
fig.show()

### Add HUP Atlas Channel Positions to 3D Plot

Loads HUP atlas channel coordinates and adds them as markers to an existing 3D plot.

- **Steps**:
  1. Load `HUP_atlas.mat` and convert `mni_coords` to `hup_df` DataFrame with `x`, `y`, and `z` columns.
  2. Add a new 3D scatter trace to `fig` for HUP channel positions, using a distinct marker size and color for visual separation.
  3. Display the updated plot with atlas coordinates, MNI, and HUP channel positions.

- **Output**: A combined interactive 3D plot showing ROI, MNI, and HUP channel positions.


In [ ]:
hup_atlas = sio.loadmat('../Data/HUP_atlas.mat')
hup_df = pd.DataFrame(hup_atlas['mni_coords'], columns=['x', 'y', 'z'])

In [ ]:
fig.add_trace(go.Scatter3d(
    x=hup_df['x'],
    y=hup_df['y'],
    z=hup_df['z'],
    mode='markers',
    marker=dict(size=5, color='white'),  # Customize size and color for distinction
    name='Electrodes'
))

# Show the combined plot
fig.show()

### Localize Coordinates to Closest ROI

Calculates the closest ROI for each electrode based on Euclidean distance and appends the closest ROI information to electrode coordinates.

- **Steps**:
  1. **Distance Calculation**: Compute pairwise Euclidean distances (`mni_dist`) between each ROI in `dkatlas_coords` and each electrode in `mni_df`.
  2. **Identify Closest ROIs**: Use `argmin` on `mni_dist` to find the closest ROI for each electrode based on the minimum distance.
  3. **Filter and Join**: Extract the closest ROI information, drop irrelevant columns, and reset the index for a clean DataFrame.
  4. **Merge**: Append the closest ROI information to `mni_df` for each electrode.

- **Output**: `mni_df` with added columns indicating the nearest ROI to each electrode.


In [ ]:
mni_dist = distance.cdist(dkatlas_coords.loc[:, ['x','y','z']], mni_df, 'euclidean')

In [ ]:
# where is the index of np min across the coloums of mni_dist where the min value is
index_mni_closest_ROI = mni_dist.argmin(axis=0)

In [ ]:
# get the closest ROI to each electrode
closest_ROI_hup = dkatlas_coords.iloc[index_mni_closest_ROI,:].copy()
closest_ROI_hup.drop(columns=['x', 'y' , 'z', 'roi', 'cmap_R', 'cmap_G', 'cmap_B'], inplace=True)
closest_ROI_hup.reset_index(drop=True, inplace=True)
mni_df = pd.DataFrame.join(mni_df, closest_ROI_hup)

### Associate Each HUP Electrode with its Closest ROI

Calculates and associates the closest ROI for each HUP electrode based on spatial distance, then appends this ROI information to `hup_df`.

- **Steps**:
  1. **Compute Distances**: Use `distance.cdist` to find the Euclidean distance between all ROIs in `dkatlas_coords` and each electrode in `hup_df`.
  2. **Identify Closest ROIs**: Obtain the index of the closest ROI for each electrode by using `argmin` on `hup_dist`.
  3. **Extract ROI Data**: Retrieve the relevant ROI data using `index_hup_closest_ROI`, and clean up by removing irrelevant columns (`x`, `y`, `z`, `roi`, `cmap_R`, `cmap_G`, `cmap_B`).
  4. **Merge**: Append the ROI information to `hup_df`, providing each electrode with its nearest ROI details.

- **Output**: `hup_df` with appended columns that describe the nearest ROI for each electrode.


In [ ]:
hup_dist = distance.cdist(dkatlas_coords.loc[:, ['x','y','z']], hup_df, 'euclidean')

In [ ]:
index_hup_closest_ROI = hup_dist.argmin(axis=0)

In [ ]:
# get the closest ROI to each electrode
closest_ROI_hup = dkatlas_coords.iloc[index_hup_closest_ROI,:].copy()
closest_ROI_hup.drop(columns=['x', 'y' , 'z', 'roi', 'cmap_R', 'cmap_G', 'cmap_B'], inplace=True)
closest_ROI_hup.reset_index(drop=True, inplace=True)
hup_df = pd.DataFrame.join(hup_df, closest_ROI_hup)

In [ ]:
# pickle hup_df and mni_df
hup_df.to_pickle('../Data/hup_df.pkl')
mni_df.to_pickle('../Data/mni_df.pkl')